In [366]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import cm
import matplotlib.pyplot as plt

# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import metnum

/home/john/Desktop/TP/tp3-metnum/notebooks
Python 3.8.3


In [367]:
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import  SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def errorRMSE(y1,y2 ):
    return sqrt ( mean_squared_error(y1,y2) )

df1h = pd.read_csv('../data/train.csv')
#df = df[df['provincia'] == 'Distrito Federal']
def polinomizar(df1h):
    df1h['metroscubiertos']  =  df1h['metroscubiertos'].apply(lambda x:  (x)    )
    df1h['metroscubiertos0']  = df1h['metroscubiertos'].apply(lambda x:  (x**0) )
    df1h['metroscubiertos2']  = df1h['metroscubiertos'].apply(lambda x:  (x**2) )
    df1h['metroscubiertos3']  = df1h['metroscubiertos'].apply(lambda x:  (x**3) )
    df1h['metroscubiertos4']  = df1h['metroscubiertos'].apply(lambda x:  (x**4) )
    df1h['metroscubiertos5']  = df1h['metroscubiertos'].apply(lambda x:  (x**5) )

    df1h['banos']  =  df1h['banos'].apply(lambda x:  (x)    )
    df1h['banos0']  = df1h['banos'].apply(lambda x:  (x**0) )
    df1h['banos2']  = df1h['banos'].apply(lambda x:  (x**2) )
    df1h['banos3']  = df1h['banos'].apply(lambda x:  (x**3) )
    df1h['banos4']  = df1h['banos'].apply(lambda x:  (x**4) )
    df1h['banos5']  = df1h['banos'].apply(lambda x:  (x**5) )

    df1h['antiguedad']  =  df1h['antiguedad'].apply(lambda x:  (x)    )
    df1h['antiguedad0']  = df1h['antiguedad'].apply(lambda x:  (x**0) )
    df1h['antiguedad2']  = df1h['antiguedad'].apply(lambda x:  (x**2) )
    df1h['antiguedad3']  = df1h['antiguedad'].apply(lambda x:  (x**3) )
    df1h['antiguedad4']  = df1h['antiguedad'].apply(lambda x:  (x**4) )
    df1h['antiguedad5']  = df1h['antiguedad'].apply(lambda x:  (x**5) )

df1h = df1h.dropna(subset = ['habitaciones'])
dfCasa = df1h.loc[( (df1h['tipodepropiedad'] == 'Casa') | (df1h['tipodepropiedad'] == 'Casa en condominio')
                        | (df1h['tipodepropiedad'] == 'Casa uso de suelo')| (df1h['tipodepropiedad'] == 'Duplex'))].copy()

dfApartamento = df1h.loc[( (df1h['tipodepropiedad'] == 'Apartamento') | (df1h['tipodepropiedad'] == 'Edificio')
                         )].copy()

dfCasa = dfCasa[['antiguedad','metroscubiertos', 'habitaciones','banos']].dropna()
dfApartamento = dfApartamento[['antiguedad','metroscubiertos','habitaciones','banos']].dropna()
df1h = df1h[['antiguedad','metroscubiertos','habitaciones','banos']].dropna()
polinomizar(dfCasa)
polinomizar(dfApartamento)
polinomizar(df1h)

# cols = list(df1h)
# cols.insert(0, cols.pop(cols.index('habitaciones')))
# df1h = df1h.loc[:, cols]
# ix = df1h.corr().abs().sort_values('habitaciones', ascending=False).index
# df1h = df1h.loc[:, ix]
# df1h.corr()

In [368]:
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.feature_selection import  SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def errorRMSE(y1,y2 ):
    return sqrt ( mean_squared_error(y1,y2) )

def errorRMSLE(y1,y2 ):
    return sqrt ( mean_squared_log_error( (y1), (y2)) )


dfApartamentoTrain = dfApartamento.loc[:, dfApartamento.columns != 'habitaciones']

X_train = dfApartamentoTrain.values
n =len(dfApartamentoTrain.columns)

y_train = dfApartamento['habitaciones'].values
precio_minimo =y_train.min()
X = X_train.reshape(len(X_train), n)
y = y_train.reshape(len(y_train),)

from sklearn.model_selection import KFold
def aplicarCota(lista, cota):
    for i, valor in enumerate(lista):
        if valor < cota:
            lista[i] = cota
#     return lista

def completar(a,b,X_test,df1h):
    def estaen(elem, X):
        for x in X:
            if(elem[0] == x[0] and elem[1] == x[1] and elem[2] == x[2]):
               return True
        return False
    X_test= np.array(X_test)
    X_train = []
    y_train = []
    dfLocal = df1h.loc[:, df1h.columns != 'habitaciones'].copy()
    X_total = dfLocal.values
    n =len(dfLocal.columns)

    y_total = df1h['habitaciones'].values
    X_total = X_total.reshape(len(X_total), n)
    y_total = y_total.reshape(len(y_total),)

    for i, x in enumerate(X_total):
       
        if not (X_test == x).all(axis=1).any():
            X_train.append(x)
            y_train.append(y_total[i])
#             print("ENTRE")
    print(len(X_train))
    return (X_train, y_train)
errorRMSEBajaDensidad = 0
errorRLMSEBajaDensidad = 0
splits = 5
kf = KFold(n_splits=splits, random_state=42, shuffle=True)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(len(X_train))
#     X_train, y_train = completar(X_train,y_train,X_test,df1h)
    lr = metnum.LinearRegression()

    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    aplicarCota(y_pred, precio_minimo)
#     print(errorRMSLE(y_test,y_pred)  )
    errorRMSEBajaDensidad += errorRMSE(y_test,y_pred) 
    errorRLMSEBajaDensidad += errorRMSLE(y_test,y_pred)  
errorRMSEBajaDensidad = (errorRMSEBajaDensidad/splits)
errorRLMSEBajaDensidad = (errorRLMSEBajaDensidad/splits)

print(errorRMSEBajaDensidad)
print(errorRLMSEBajaDensidad)
print(y.mean() )

37316
37316
37316
37316
37316
0.5127553945572187
0.15458653990259003
2.3906528030871477


In [369]:
dfCasaTrain = dfCasa.loc[:, dfCasa.columns != 'habitaciones']

X_train = dfCasaTrain.values
n =len(dfCasaTrain.columns)

y_train = dfCasa['habitaciones'].values
precio_minimo =y_train.min()
X = X_train.reshape(len(X_train), n)
y = y_train.reshape(len(y_train),)

from sklearn.model_selection import KFold
def aplicarCota(lista, cota):
    for i, valor in enumerate(lista):
        if valor < cota:
            lista[i] = cota


errorRMSEMuchaDensidad = 0
errorRLMSEMuchaDensidad = 0
splits = 5
kf = KFold(n_splits=splits, random_state=42, shuffle=True)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#     X_train, y_train = completar(X_train,y_train,X_test,df1h)

    lr = metnum.LinearRegression()

    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    aplicarCota(y_pred, precio_minimo)
    errorRMSEMuchaDensidad += errorRMSE(y_test,y_pred) 
    errorRLMSEMuchaDensidad += errorRMSLE(y_test,y_pred)  
    
    
errorRMSEMuchaDensidad=(errorRMSEMuchaDensidad/splits)
errorRLMSEMuchaDensidad=(errorRLMSEMuchaDensidad/splits)
print(errorRMSEMuchaDensidad)
print(errorRLMSEMuchaDensidad)
print(y.mean())

0.6569735953458664
0.15121008162486443
3.027318737468469


In [370]:
#Promedio de errores
# print( (errorRMSEMuchaDensidad +errorRMSEBajaDensidad)/2)
# print( (errorRLMSEMuchaDensidad +errorRLMSEBajaDensidad)/2)

# 0.5852154317971143
# 0.15302261494798855


In [371]:
# from math import sqrt
# from sklearn.metrics import mean_squared_error
# from sklearn.feature_selection import  SelectKBest, f_regression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler

# def errorRMSE(y1,y2 ):
#     return sqrt ( mean_squared_error(y1,y2) )

# df1h = pd.read_csv('../data/train.csv')
# #df = df[df['provincia'] == 'Distrito Federal']
# def polinomizar(df1h):
#     df1h['metroscubiertos']  =  df1h['metroscubiertos'].apply(lambda x:  (x)    )
# #     df1h['metroscubiertos0']  = df1h['metroscubiertos'].apply(lambda x:  (x**0) )
# #     df1h['metroscubiertos2']  = df1h['metroscubiertos'].apply(lambda x:  (x**2) )
# #     df1h['metroscubiertos3']  = df1h['metroscubiertos'].apply(lambda x:  (x**3) )
# #     df1h['metroscubiertos4']  = df1h['metroscubiertos'].apply(lambda x:  (x**4) )

#     df1h['banos']  =  df1h['banos'].apply(lambda x:  (x)    )
# #     df1h['banos0']  = df1h['banos'].apply(lambda x:  (x**0) )
# #     df1h['banos2']  = df1h['banos'].apply(lambda x:  (x**2) )
# #     df1h['banos3']  = df1h['banos'].apply(lambda x:  (x**3) )
# #     df1h['banos4']  = df1h['banos'].apply(lambda x:  (x**4) )

#     df1h['antiguedad']  =  df1h['antiguedad'].apply(lambda x:  (x)    )
# #     df1h['antiguedad0']  = df1h['antiguedad'].apply(lambda x:  (x**0) )
# #     df1h['antiguedad2']  = df1h['antiguedad'].apply(lambda x:  (x**2) )
# #     df1h['antiguedad3']  = df1h['antiguedad'].apply(lambda x:  (x**3) )
# #     df1h['antiguedad4']  = df1h['antiguedad'].apply(lambda x:  (x**4) )

# df1h = df1h.dropna(subset = ['habitaciones'])
# dfCasa = df1h.copy()

# dfCasa = dfCasa[['antiguedad','metroscubiertos','banos','habitaciones']].dropna()
# polinomizar(dfCasa)

# dfCasaTrain = dfCasa.loc[:, dfCasa.columns != 'habitaciones']

# X_train = dfCasaTrain.values
# n =len(dfCasaTrain.columns)

# y_train = dfCasa['habitaciones'].values
# precio_minimo =y_train.min()
# X = X_train.reshape(len(X_train), n)
# y = y_train.reshape(len(y_train),)

# from sklearn.model_selection import KFold
# def aplicarCota(lista, cota):
#     for i, valor in enumerate(lista):
#         if valor < cota:
#             lista[i] = cota


# errorRMSEMuchaDensidad = 0
# errorRLMSEMuchaDensidad = 0
# splits = 5
# kf = KFold(n_splits=splits, random_state=42, shuffle=True)
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     lr = metnum.LinearRegression()

#     lr.fit(X_train, y_train)
#     y_pred = lr.predict(X_test)
#     aplicarCota(y_pred, precio_minimo)
#     errorRMSEMuchaDensidad += errorRMSE(y_test,y_pred) 
#     errorRLMSEMuchaDensidad += errorRMSLE(y_test,y_pred)  
    
    
# errorRMSEMuchaDensidad=(errorRMSEMuchaDensidad/splits)
# errorRLMSEMuchaDensidad=(errorRLMSEMuchaDensidad/splits)
# print(errorRMSEMuchaDensidad)
# print(errorRLMSEMuchaDensidad)